In [1]:
## This file implements neural networks and random forest on p0006presabs_qual.
## Due to the imbalanced data, we implement both over-sampling method and the combination of under- and over-sampling method.
## For fully-connected neural networks, the accuracy is 70.09% for over-sampling data after improvement, 
## 93.33% for combination data and 88.89% with dropout.
## For random forest, the accuracy is 65.81% for over-sampling data and 95.56% for combination data.
## For random forest with cross-validation, the mean accuracy is 67.1% for over-sampling, and 97.95% for combination data.

In [2]:
from numpy.random import seed
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [3]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/phage_qual/p0006presabs_qual.csv')
df.shape

(255, 86)

In [4]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [5]:
df['pheno']

0      1
1      0
2      1
3      1
4      1
5      1
6      1
7      0
8      1
9      1
10     0
11     2
12     1
13     0
14     0
15     0
16     1
17     1
18     2
19     0
20     1
21     1
22     0
23     0
24     2
25     0
26     2
27     0
28     0
29     2
      ..
225    1
226    1
227    1
228    2
229    2
230    1
231    0
232    1
233    1
234    1
235    2
236    0
237    0
238    0
239    0
240    0
241    1
242    0
243    0
244    1
245    0
246    1
247    0
248    0
249    0
250    1
251    0
252    0
253    0
254    0
Name: pheno, Length: 255, dtype: int64

In [6]:
df['pheno'].value_counts()

0    130
1     86
2     39
Name: pheno, dtype: int64

In [7]:
df.head()

,id,TTGTTATAGTC,TTAATTTAATAGA,TTAACATAATAAT,TGCAATCTCTTTAT,TATTATGTTAATG,TACATACCGAT,GTGTATCATAAT,GCTGTTGAAATGGC,GCAAACATGCG,...,group_7850,group_7871,group_10187,group_10190,group_4016,group_6375,group_7207,group_8240,group_9605,pheno
0,107,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
1,109,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,115,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
3,120335,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
4,120337,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1


In [8]:
df_clean = df.drop(columns=['id'])

In [9]:
df_clean.shape

(255, 85)

In [10]:
df_clean.head()

,TTGTTATAGTC,TTAATTTAATAGA,TTAACATAATAAT,TGCAATCTCTTTAT,TATTATGTTAATG,TACATACCGAT,GTGTATCATAAT,GCTGTTGAAATGGC,GCAAACATGCG,GAGTCCTGTT,...,group_7850,group_7871,group_10187,group_10190,group_4016,group_6375,group_7207,group_8240,group_9605,pheno
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
3,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
4,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1


In [11]:
X = df_clean.loc[:, df_clean.columns != 'pheno'].values
y = df_clean['pheno'].values
print(X.shape, y.shape)

(255, 84) (255,)


In [12]:
# over-sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

[(0, 130), (1, 130), (2, 130)]


Using TensorFlow backend.


In [13]:
# combination of under- and over- sampling
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=100)
X_comb, y_comb = smote_enn.fit_resample(X, y)
print(sorted(Counter(y_comb).items()))

[(0, 68), (1, 12), (2, 67)]


In [14]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [15]:
# split into train, validation, and test data (combination)
from sklearn.model_selection import train_test_split
X_train_comb, X_test_comb, y_train_comb, y_test_comb = train_test_split(X_comb, y_comb,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_comb)

In [16]:
############# Fully-Connected Neural Network ################

In [17]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
import numpy as np

In [18]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [19]:
model1_over.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [20]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 273 samples, validate on 117 samples
Epoch 1/100
273/273 [==============================] - 0s 1ms/step - loss: 1.1098 - accuracy: 0.3297 - val_loss: 1.0402 - val_accuracy: 0.4957
Epoch 2/100
273/273 [==============================] - 0s 159us/step - loss: 1.0284 - accuracy: 0.5092 - val_loss: 0.9997 - val_accuracy: 0.5214
Epoch 3/100
273/273 [==============================] - 0s 102us/step - loss: 0.9747 - accuracy: 0.5934 - val_loss: 0.9674 - val_accuracy: 0.5556
Epoch 4/100
273/273 [==============================] - 0s 104us/step - loss: 0.9367 - accuracy: 0.6044 - val_loss: 0.9423 - val_accuracy: 0.5470
Epoch 5/100
273/273 [==============================] - 0s 98us/step - loss: 0.9028 - accuracy: 0.6227 - val_loss: 0.9192 - val_accuracy: 0.5726
Epoch 6/100
273/273 [==============================] - 0s 127us/step - loss: 0.8773 - accuracy: 0.6447 - val_loss: 0.9041 - val_accuracy: 0.5812
Epoch 7/100
273/273 [==============================] - 0s 121us/step - loss: 0.8548 - a

Epoch 57/100
273/273 [==============================] - 0s 102us/step - loss: 0.5639 - accuracy: 0.7619 - val_loss: 0.7443 - val_accuracy: 0.6496
Epoch 58/100
273/273 [==============================] - 0s 102us/step - loss: 0.5635 - accuracy: 0.7766 - val_loss: 0.7456 - val_accuracy: 0.6496
Epoch 59/100
273/273 [==============================] - 0s 87us/step - loss: 0.5605 - accuracy: 0.7619 - val_loss: 0.7363 - val_accuracy: 0.6581
Epoch 60/100
273/273 [==============================] - 0s 85us/step - loss: 0.5576 - accuracy: 0.7692 - val_loss: 0.7377 - val_accuracy: 0.6496
Epoch 61/100
273/273 [==============================] - 0s 125us/step - loss: 0.5527 - accuracy: 0.7766 - val_loss: 0.7416 - val_accuracy: 0.6667
Epoch 62/100
273/273 [==============================] - 0s 206us/step - loss: 0.5531 - accuracy: 0.7802 - val_loss: 0.7363 - val_accuracy: 0.6496
Epoch 63/100
273/273 [==============================] - 0s 167us/step - loss: 0.5486 - accuracy: 0.7766 - val_loss: 0.7400 - v

In [21]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

117/117 [==============================] - 0s 57us/step
over-sampling test accuracy: 68.38%


In [22]:
#### improve neural network on over-sampling data
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
])

In [23]:
model1_over2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [24]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 273 samples, validate on 117 samples
Epoch 1/1000
273/273 [==============================] - 0s 499us/step - loss: 1.1804 - accuracy: 0.3443 - val_loss: 1.1196 - val_accuracy: 0.3761
Epoch 2/1000
273/273 [==============================] - 0s 87us/step - loss: 1.0817 - accuracy: 0.4615 - val_loss: 1.0645 - val_accuracy: 0.4615
Epoch 3/1000
273/273 [==============================] - 0s 84us/step - loss: 1.0353 - accuracy: 0.5092 - val_loss: 1.0330 - val_accuracy: 0.4444
Epoch 4/1000
273/273 [==============================] - 0s 86us/step - loss: 0.9996 - accuracy: 0.5531 - val_loss: 1.0082 - val_accuracy: 0.5043
Epoch 5/1000
273/273 [==============================] - 0s 82us/step - loss: 0.9704 - accuracy: 0.5824 - val_loss: 0.9827 - val_accuracy: 0.5043
Epoch 6/1000
273/273 [==============================] - 0s 81us/step - loss: 0.9370 - accuracy: 0.6081 - val_loss: 0.9595 - val_accuracy: 0.5043
Epoch 7/1000
273/273 [==============================] - 0s 90us/step - loss: 0.9076

Epoch 57/1000
273/273 [==============================] - 0s 79us/step - loss: 0.4783 - accuracy: 0.7875 - val_loss: 0.7165 - val_accuracy: 0.7094
Epoch 58/1000
273/273 [==============================] - 0s 71us/step - loss: 0.4790 - accuracy: 0.7912 - val_loss: 0.7044 - val_accuracy: 0.7179
Epoch 59/1000
273/273 [==============================] - 0s 74us/step - loss: 0.4785 - accuracy: 0.7949 - val_loss: 0.7237 - val_accuracy: 0.6752
Epoch 60/1000
273/273 [==============================] - 0s 75us/step - loss: 0.4712 - accuracy: 0.7949 - val_loss: 0.7131 - val_accuracy: 0.7179
Epoch 61/1000
273/273 [==============================] - 0s 75us/step - loss: 0.4722 - accuracy: 0.7802 - val_loss: 0.7204 - val_accuracy: 0.6752
Epoch 62/1000
273/273 [==============================] - 0s 75us/step - loss: 0.4632 - accuracy: 0.7802 - val_loss: 0.7062 - val_accuracy: 0.7179
Epoch 63/1000
273/273 [==============================] - 0s 82us/step - loss: 0.4628 - accuracy: 0.7912 - val_loss: 0.6989 -

273/273 [==============================] - 0s 77us/step - loss: 0.3881 - accuracy: 0.8095 - val_loss: 0.7476 - val_accuracy: 0.6752
Epoch 114/1000
273/273 [==============================] - 0s 85us/step - loss: 0.3906 - accuracy: 0.8278 - val_loss: 0.7567 - val_accuracy: 0.6667
Epoch 115/1000
273/273 [==============================] - 0s 76us/step - loss: 0.3878 - accuracy: 0.8205 - val_loss: 0.7403 - val_accuracy: 0.6752
Epoch 116/1000
273/273 [==============================] - 0s 71us/step - loss: 0.3882 - accuracy: 0.8278 - val_loss: 0.7550 - val_accuracy: 0.6667
Epoch 117/1000
273/273 [==============================] - 0s 79us/step - loss: 0.3892 - accuracy: 0.8278 - val_loss: 0.7347 - val_accuracy: 0.7094
Epoch 118/1000
273/273 [==============================] - 0s 78us/step - loss: 0.3818 - accuracy: 0.8278 - val_loss: 0.7490 - val_accuracy: 0.6923
Epoch 119/1000
273/273 [==============================] - 0s 78us/step - loss: 0.3906 - accuracy: 0.8205 - val_loss: 0.7377 - val_acc

Epoch 169/1000
273/273 [==============================] - 0s 73us/step - loss: 0.3604 - accuracy: 0.8315 - val_loss: 0.7735 - val_accuracy: 0.6752
Epoch 170/1000
273/273 [==============================] - 0s 73us/step - loss: 0.3593 - accuracy: 0.8315 - val_loss: 0.7814 - val_accuracy: 0.6752
Epoch 171/1000
273/273 [==============================] - 0s 71us/step - loss: 0.3599 - accuracy: 0.8242 - val_loss: 0.7646 - val_accuracy: 0.6581
Epoch 172/1000
273/273 [==============================] - 0s 71us/step - loss: 0.3569 - accuracy: 0.8352 - val_loss: 0.7856 - val_accuracy: 0.6752
Epoch 173/1000
273/273 [==============================] - 0s 86us/step - loss: 0.3581 - accuracy: 0.8352 - val_loss: 0.7874 - val_accuracy: 0.6581
Epoch 174/1000
273/273 [==============================] - 0s 85us/step - loss: 0.3567 - accuracy: 0.8168 - val_loss: 0.7992 - val_accuracy: 0.6752
Epoch 175/1000
273/273 [==============================] - 0s 76us/step - loss: 0.3602 - accuracy: 0.8315 - val_loss: 0

Epoch 225/1000
273/273 [==============================] - 0s 77us/step - loss: 0.3470 - accuracy: 0.8242 - val_loss: 0.8166 - val_accuracy: 0.6923
Epoch 226/1000
273/273 [==============================] - 0s 78us/step - loss: 0.3441 - accuracy: 0.8315 - val_loss: 0.8116 - val_accuracy: 0.6923
Epoch 227/1000
273/273 [==============================] - 0s 79us/step - loss: 0.3431 - accuracy: 0.8388 - val_loss: 0.8098 - val_accuracy: 0.6667
Epoch 228/1000
273/273 [==============================] - 0s 78us/step - loss: 0.3467 - accuracy: 0.8278 - val_loss: 0.8140 - val_accuracy: 0.6496
Epoch 229/1000
273/273 [==============================] - 0s 75us/step - loss: 0.3528 - accuracy: 0.8205 - val_loss: 0.8431 - val_accuracy: 0.6154
Epoch 230/1000
273/273 [==============================] - 0s 76us/step - loss: 0.3483 - accuracy: 0.8278 - val_loss: 0.8033 - val_accuracy: 0.7009
Epoch 231/1000
273/273 [==============================] - 0s 77us/step - loss: 0.3420 - accuracy: 0.8315 - val_loss: 0

...

Epoch 727/1000
273/273 [==============================] - 0s 98us/step - loss: 0.3291 - accuracy: 0.8242 - val_loss: 0.9442 - val_accuracy: 0.6667
Epoch 728/1000
273/273 [==============================] - 0s 153us/step - loss: 0.3408 - accuracy: 0.8205 - val_loss: 0.9537 - val_accuracy: 0.6667
Epoch 729/1000
273/273 [==============================] - 0s 132us/step - loss: 0.3313 - accuracy: 0.8425 - val_loss: 0.9740 - val_accuracy: 0.6581
Epoch 730/1000
273/273 [==============================] - 0s 80us/step - loss: 0.3324 - accuracy: 0.8278 - val_loss: 0.9319 - val_accuracy: 0.6923
Epoch 731/1000
273/273 [==============================] - 0s 84us/step - loss: 0.3346 - accuracy: 0.8242 - val_loss: 0.9277 - val_accuracy: 0.6752
Epoch 732/1000
273/273 [==============================] - 0s 85us/step - loss: 0.3305 - accuracy: 0.8132 - val_loss: 0.9402 - val_accuracy: 0.6838
Epoch 733/1000
273/273 [==============================] - 0s 77us/step - loss: 0.3368 - accuracy: 0.8315 - val_loss:

273/273 [==============================] - 0s 77us/step - loss: 0.3291 - accuracy: 0.8278 - val_loss: 0.9512 - val_accuracy: 0.6923
Epoch 783/1000
273/273 [==============================] - 0s 74us/step - loss: 0.3328 - accuracy: 0.8352 - val_loss: 0.9590 - val_accuracy: 0.6923
Epoch 784/1000
273/273 [==============================] - 0s 73us/step - loss: 0.3284 - accuracy: 0.8168 - val_loss: 0.9517 - val_accuracy: 0.6923
Epoch 785/1000
273/273 [==============================] - 0s 101us/step - loss: 0.3303 - accuracy: 0.8095 - val_loss: 0.9435 - val_accuracy: 0.6838
Epoch 786/1000
273/273 [==============================] - 0s 74us/step - loss: 0.3323 - accuracy: 0.8315 - val_loss: 0.9616 - val_accuracy: 0.6581
Epoch 787/1000
273/273 [==============================] - 0s 73us/step - loss: 0.3279 - accuracy: 0.8352 - val_loss: 0.9284 - val_accuracy: 0.6923
Epoch 788/1000
273/273 [==============================] - 0s 73us/step - loss: 0.3317 - accuracy: 0.8278 - val_loss: 0.9562 - val_ac

Epoch 838/1000
273/273 [==============================] - 0s 82us/step - loss: 0.3327 - accuracy: 0.8352 - val_loss: 0.9730 - val_accuracy: 0.6667
Epoch 839/1000
273/273 [==============================] - 0s 70us/step - loss: 0.3294 - accuracy: 0.8315 - val_loss: 0.9655 - val_accuracy: 0.6752
Epoch 840/1000
273/273 [==============================] - 0s 79us/step - loss: 0.3289 - accuracy: 0.8205 - val_loss: 0.9725 - val_accuracy: 0.6581
Epoch 841/1000
273/273 [==============================] - 0s 82us/step - loss: 0.3302 - accuracy: 0.8242 - val_loss: 0.9652 - val_accuracy: 0.6923
Epoch 842/1000
273/273 [==============================] - 0s 81us/step - loss: 0.3329 - accuracy: 0.8352 - val_loss: 0.9639 - val_accuracy: 0.6581
Epoch 843/1000
273/273 [==============================] - 0s 80us/step - loss: 0.3348 - accuracy: 0.8315 - val_loss: 0.9788 - val_accuracy: 0.6581
Epoch 844/1000
273/273 [==============================] - 0s 73us/step - loss: 0.3313 - accuracy: 0.8388 - val_loss: 0

273/273 [==============================] - 0s 144us/step - loss: 0.3311 - accuracy: 0.8168 - val_loss: 0.9463 - val_accuracy: 0.6838
Epoch 894/1000
273/273 [==============================] - 0s 91us/step - loss: 0.3358 - accuracy: 0.8278 - val_loss: 0.9877 - val_accuracy: 0.6667
Epoch 895/1000
273/273 [==============================] - 0s 76us/step - loss: 0.3325 - accuracy: 0.8315 - val_loss: 0.9547 - val_accuracy: 0.6838
Epoch 896/1000
273/273 [==============================] - 0s 75us/step - loss: 0.3303 - accuracy: 0.8315 - val_loss: 0.9855 - val_accuracy: 0.6581
Epoch 897/1000
273/273 [==============================] - 0s 72us/step - loss: 0.3313 - accuracy: 0.8352 - val_loss: 0.9937 - val_accuracy: 0.6581
Epoch 898/1000
273/273 [==============================] - 0s 84us/step - loss: 0.3300 - accuracy: 0.8315 - val_loss: 0.9717 - val_accuracy: 0.7265
Epoch 899/1000
273/273 [==============================] - 0s 70us/step - loss: 0.3348 - accuracy: 0.8132 - val_loss: 1.0069 - val_ac

273/273 [==============================] - 0s 81us/step - loss: 0.3292 - accuracy: 0.8242 - val_loss: 0.9591 - val_accuracy: 0.6752
Epoch 949/1000
273/273 [==============================] - 0s 74us/step - loss: 0.3269 - accuracy: 0.8315 - val_loss: 0.9843 - val_accuracy: 0.6496
Epoch 950/1000
273/273 [==============================] - 0s 73us/step - loss: 0.3318 - accuracy: 0.8352 - val_loss: 1.0040 - val_accuracy: 0.6496
Epoch 951/1000
273/273 [==============================] - 0s 81us/step - loss: 0.3287 - accuracy: 0.8352 - val_loss: 0.9463 - val_accuracy: 0.6923
Epoch 952/1000
273/273 [==============================] - 0s 79us/step - loss: 0.3394 - accuracy: 0.8352 - val_loss: 0.9543 - val_accuracy: 0.6752
Epoch 953/1000
273/273 [==============================] - 0s 76us/step - loss: 0.3385 - accuracy: 0.8278 - val_loss: 0.9922 - val_accuracy: 0.6923
Epoch 954/1000
273/273 [==============================] - 0s 97us/step - loss: 0.3372 - accuracy: 0.8315 - val_loss: 0.9622 - val_acc

In [25]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

117/117 [==============================] - 0s 55us/step
over-sampling test accuracy: 70.09%


In [26]:
#### neural network on combination data
model1_comb = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],)),
    Dense(3, activation='softmax'),
])

In [27]:
model1_comb.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [28]:
model1_comb.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=100,
          validation_data=(X_test_comb, y_test_comb))

Train on 102 samples, validate on 45 samples
Epoch 1/100
102/102 [==============================] - 0s 1ms/step - loss: 1.0043 - accuracy: 0.4804 - val_loss: 0.8877 - val_accuracy: 0.6444
Epoch 2/100
102/102 [==============================] - 0s 261us/step - loss: 0.8844 - accuracy: 0.5882 - val_loss: 0.7942 - val_accuracy: 0.7556
Epoch 3/100
102/102 [==============================] - 0s 285us/step - loss: 0.7947 - accuracy: 0.7941 - val_loss: 0.7213 - val_accuracy: 0.8667
Epoch 4/100
102/102 [==============================] - 0s 404us/step - loss: 0.7200 - accuracy: 0.9020 - val_loss: 0.6673 - val_accuracy: 0.8667
Epoch 5/100
102/102 [==============================] - 0s 300us/step - loss: 0.6630 - accuracy: 0.9118 - val_loss: 0.6247 - val_accuracy: 0.8889
Epoch 6/100
102/102 [==============================] - 0s 354us/step - loss: 0.6177 - accuracy: 0.9118 - val_loss: 0.5911 - val_accuracy: 0.8889
Epoch 7/100
102/102 [==============================] - 0s 369us/step - loss: 0.5826 - a

Epoch 57/100
102/102 [==============================] - 0s 338us/step - loss: 0.1372 - accuracy: 0.9216 - val_loss: 0.2379 - val_accuracy: 0.8889
Epoch 58/100
102/102 [==============================] - 0s 305us/step - loss: 0.1352 - accuracy: 0.9216 - val_loss: 0.2361 - val_accuracy: 0.8889
Epoch 59/100
102/102 [==============================] - 0s 283us/step - loss: 0.1331 - accuracy: 0.9216 - val_loss: 0.2359 - val_accuracy: 0.8889
Epoch 60/100
102/102 [==============================] - 0s 407us/step - loss: 0.1306 - accuracy: 0.9216 - val_loss: 0.2336 - val_accuracy: 0.9333
Epoch 61/100
102/102 [==============================] - 0s 452us/step - loss: 0.1283 - accuracy: 0.9706 - val_loss: 0.2311 - val_accuracy: 0.9333
Epoch 62/100
102/102 [==============================] - 0s 353us/step - loss: 0.1262 - accuracy: 0.9706 - val_loss: 0.2272 - val_accuracy: 0.9333
Epoch 63/100
102/102 [==============================] - 0s 328us/step - loss: 0.1241 - accuracy: 0.9706 - val_loss: 0.2252 -

In [29]:
acc_test_comb = model1_comb.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb*100))

45/45 [==============================] - 0s 312us/step
combination test accuracy: 95.56%


In [30]:
#### add dropout
model1_comb2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],)),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [31]:
model1_comb2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [32]:
model1_comb2.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=100,
          validation_data=(X_test_comb, y_test_comb))

Train on 102 samples, validate on 45 samples
Epoch 1/100
102/102 [==============================] - 0s 3ms/step - loss: 3.7119 - accuracy: 0.4804 - val_loss: 0.8336 - val_accuracy: 0.6222
Epoch 2/100
102/102 [==============================] - 0s 196us/step - loss: 3.7909 - accuracy: 0.5196 - val_loss: 0.7405 - val_accuracy: 0.6889
Epoch 3/100
102/102 [==============================] - 0s 176us/step - loss: 3.1649 - accuracy: 0.5588 - val_loss: 0.6794 - val_accuracy: 0.7111
Epoch 4/100
102/102 [==============================] - 0s 179us/step - loss: 4.3359 - accuracy: 0.5588 - val_loss: 0.6341 - val_accuracy: 0.7778
Epoch 5/100
102/102 [==============================] - 0s 212us/step - loss: 3.1304 - accuracy: 0.5882 - val_loss: 0.5966 - val_accuracy: 0.8667
Epoch 6/100
102/102 [==============================] - 0s 172us/step - loss: 2.9929 - accuracy: 0.7255 - val_loss: 0.5666 - val_accuracy: 0.9111
Epoch 7/100
102/102 [==============================] - 0s 158us/step - loss: 3.3271 - a

Epoch 57/100
102/102 [==============================] - 0s 338us/step - loss: 1.9089 - accuracy: 0.7451 - val_loss: 0.4637 - val_accuracy: 0.9111
Epoch 58/100
102/102 [==============================] - 0s 434us/step - loss: 1.5156 - accuracy: 0.7451 - val_loss: 0.4674 - val_accuracy: 0.8889
Epoch 59/100
102/102 [==============================] - 0s 351us/step - loss: 2.2308 - accuracy: 0.7353 - val_loss: 0.4676 - val_accuracy: 0.8889
Epoch 60/100
102/102 [==============================] - 0s 294us/step - loss: 2.2879 - accuracy: 0.6961 - val_loss: 0.4648 - val_accuracy: 0.9111
Epoch 61/100
102/102 [==============================] - 0s 399us/step - loss: 1.6802 - accuracy: 0.7059 - val_loss: 0.4633 - val_accuracy: 0.9111
Epoch 62/100
102/102 [==============================] - 0s 176us/step - loss: 1.0711 - accuracy: 0.7941 - val_loss: 0.4517 - val_accuracy: 0.9111
Epoch 63/100
102/102 [==============================] - 0s 149us/step - loss: 1.4206 - accuracy: 0.7059 - val_loss: 0.4345 -

In [33]:
acc_test_comb2 = model1_comb2.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb2*100))

45/45 [==============================] - 0s 463us/step
combination test accuracy: 88.89%


In [34]:
############## Random Forest ##############

In [35]:
###### random forest on over-sampling data
from sklearn.ensemble import RandomForestClassifier

rf_over = RandomForestClassifier(n_estimators=100)
rf_over.fit(X_train_over,y_train_over)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [36]:
from sklearn.metrics import accuracy_score
y_pred_over = rf_over.predict(X_test_over)
print('over-sampling test accuracy: %.2f%%' % (accuracy_score(y_test_over, y_pred_over)*100))

over-sampling test accuracy: 64.96%


In [37]:
###### random forest on combination data
from sklearn.ensemble import RandomForestClassifier

rf_comb = RandomForestClassifier(n_estimators=100)
rf_comb.fit(X_train_comb,y_train_comb)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [38]:
from sklearn.metrics import accuracy_score
y_pred_comb = rf_comb.predict(X_test_comb)
print('combination test accuracy: %.2f%%' % (accuracy_score(y_test_comb, y_pred_comb)*100))

combination test accuracy: 95.56%


In [41]:
## random forest model with CV on over-sampling data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# rfcv_over = RandomForestClassifier(n_estimators=100, random_state=123)

accs_over = cross_val_score(estimator=rf_over, X=X_train_over, y=y_train_over, cv=5)
print(accs_over)
print(accs_over.mean())

[0.61403509 0.68518519 0.66666667 0.66666667 0.72222222]
0.6709551656920077


In [42]:
## random forest model with CV on combination data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# rfcv_comb = RandomForestClassifier(n_estimators=100, random_state=123)

accs_comb = cross_val_score(estimator=rf_comb, X=X_train_comb, y=y_train_comb, cv=5)
print(accs_comb)
print(accs_comb.mean())

[1.         1.         0.95       1.         0.94736842]
0.9794736842105264
